# 3-Pretrain

预训练是模型经历的第一个阶段，在该阶段，模型将会吸收知识，学习尽可能正确的下一词语预测范式。在这个笔记本中，我们仅对预训练的训练流程进行展示和学习，因此只给出必要的代码片段，如 wandb 和 ddp 不会在此笔记本中涉及。

此笔记本的完整实现见主仓库 `/minimind/train_pretrain.py`

In [1]:
# 导入依赖
import os
import platform
import argparse
import time
import math
import warnings
import pandas as pd
import torch
import torch.distributed as dist
from torch import optim, nn
from torch.nn.parallel import DistributedDataParallel
from torch.optim.lr_scheduler import CosineAnnealingLR
from torch.utils.data import DataLoader, DistributedSampler
from contextlib import nullcontext

from transformers import AutoTokenizer

from model.model import MiniMindLM
from model.LMConfig import LMConfig
from model.dataset import PretrainDataset

In [2]:
warnings.filterwarnings('ignore')

## 可选参数设置

首先，查看训练的可选参数，这些参数在实际使用时通过解析命令行进行导入，我们用 class 进行包装.

In [3]:
class args:
    epochs: int = 5 # 训练轮数 只做预训练实验 设置为 5 轮
    batch_size: int = 2 # pretrain 数据集仅两个样本，设置 batch 为 2
    learning_rate: float = 5e-4 # 学习率
    device: str = 'cuda' if torch.cuda.is_available() else 'cpu'
    dtype: str = 'bfloat16' # 16 bit 浮点数：8 bit 指数 + 7 bit 尾数
    # use_wandb: bool = False # 是否使用 wandb 我们不使用
    wandb_project: str = 'MiniMind-Notebook'
    num_workers: int = 1 # 工作进程数
    # ddp：bool = False # 单机多卡
    accumulation_steps: int = 1 # 梯度累积步数，用途是在显存受限的情况下模拟更大的 batch size
    grad_clip: float = 1.0 # 梯度剪裁
    warmup_iters: int = 0 # 学习率热启动
    log_interval: int = 1 # 每一步打印日志 仅用于观察
    local_rank: int = 1 # device 设备号
    dim: int = 512 # 词嵌入维度 模型超参数
    n_layers: int = 2 # MiniMind Block 数量 模型超参数
    max_seq_len: int = 512 # 序列长度阈值
    use_moe: bool = False # 是否启用混合专家
    data_path: str = './toydata/pretrain_data.jsonl' # 数据集路径
    save_dir: str = "./output"  # 模型保存目录
    save_weight: str = "minimind_pretrain"  # checkpoint 文件前缀
    save_interval: int = 1  # 每多少步保存一次模型，0表示不保存 我们这里只展示训练过程 （可选择的保存模型，建议先保存）

In [4]:
print(f'查看工作设备 {args.device}')

查看工作设备 cuda


## 初始化训练


接下来，我们对一些重要模块进行初始化，我们已经了解过，分词器，模型和数据集是大模型的基本组件，我们对其进行初始化。

In [5]:
def init_model(lm_config):
    tokenizer = AutoTokenizer.from_pretrained('./model/minimind_tokenizer')
    model = MiniMindLM(lm_config).to(args.device)  # 初始化模型并移动到指定设备
    print(f'LLM总参数量：{sum(p.numel() for p in model.parameters() if p.requires_grad) / 1e6:.3f} 百万')
    return model, tokenizer

In [6]:
lm_config = LMConfig(dim=args.dim, n_layers=args.n_layers, max_seq_len=args.max_seq_len, use_moe=args.use_moe)
model, tokenizer = init_model(lm_config)  # 初始化模型和分词器

# 创建预训练数据集实例
# 得到的结果是id列表，长度不超过 max_seq_len，超过部分会被截断
train_ds = PretrainDataset(args.data_path, tokenizer, max_length=lm_config.max_seq_len)  

train_loader = DataLoader(
    train_ds,
    batch_size=args.batch_size,
    pin_memory=True,  # 提高数据传输效率
    drop_last=False,  # 如果数据集大小不能被 batch_size 整除，丢弃最后一个不完整的 batch
    shuffle=False,  # 预训练数据集较小，不进行 shuffle
    num_workers=args.num_workers,  # 使用的子进程数
)

print(f'模型位于设备：{model.device}, 词表长度：{tokenizer.vocab_size}, DataLoader：{train_loader}')

LLM总参数量：8.915 百万
模型位于设备：cuda:0, 词表长度：6400, DataLoader：<torch.utils.data.dataloader.DataLoader object at 0x0000011847CD0CD0>


In [7]:
loader = iter(train_loader) 
print(f'打印一个 iter 的数据:\n{next(loader)}\n')
print(f'数据集大小：{len(train_ds)}, DataLoader 大小：{len(loader)}')

打印一个 iter 的数据:
[tensor([[   1,   46,   46,  ...,    0,    0,    0],
        [   1, 5349, 1619,  ...,    0,    0,    0]]), tensor([[  46,   46,   47,  ...,    0,    0,    0],
        [5349, 1619, 2875,  ...,    0,    0,    0]]), tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])]

数据集大小：2, DataLoader 大小：1


我们发现，train loader 的每一个 iter 都包含一个长度为 3 的张量列表，这是因为 train_dataset 每一次取数据都会返回三个张量，分别为:

- 样本 X: 包含 \<bos> 在内的输入 content
- 标签 Y: 包含 \<eos> 在内的输出 content
- 掩码 loss_mask: 指示需要计算损失的 token 位置

由于我们的数据集只有两条数据，而 batch size 设置为 2，因此我们的 dataloader 只有一个 iter.

## 启动训练

训练一个深度学习模型，还涉及到了优化器，损失函数和学习率调度。接下来，我们查看 MiniMind 训练部分的代码，并进行一轮简单的训练。

---

#### 学习率
采用余弦退火学习率调度，即余弦函数平滑衰减 + 固定下限保护，公式如下：
$$
\text{lr}_{\text{current}} = \frac{\text{lr}}{10} + 0.5 \cdot \text{lr} \cdot \left(1 + \cos\left(\pi \cdot \frac{\text{current\_step}}{\text{total\_steps}}\right)\right)
$$

#### AdamW 更新公式
1. 矩更新：
$$m_t = \beta_1 m_{t-1} + (1-\beta_1)g_t,\quad v_t = \beta_2 v_{t-1} + (1-\beta_2)g_t^2$$

2. 偏差修正：
$$\hat{m}_t = \frac{m_t}{1-\beta_1^t},\quad \hat{v}_t = \frac{v_t}{1-\beta_2^t}$$

3. 核心参数更新：
$$\theta_t = \theta_{t-1}(1-\alpha\lambda) - \alpha \frac{\hat{m}_t}{\sqrt{\hat{v}_t}+\epsilon}$$




---

In [8]:
# 学习率调度方面 采用余弦退火学习率
def get_lr(current_step, total_steps, lr):
    return lr / 10 + 0.5 * lr * (1 + math.cos(math.pi * current_step / total_steps))

# 优化器方面 选择 AdamW 优化器 并在混精度场景下创建 scaler 进行梯度缩放避免数值下溢
scaler = torch.amp.GradScaler('cuda', enabled=(args.dtype in ['float16', 'bfloat16']))  # 专门解决混合精度训练中的数值下溢问题
optimizer = optim.AdamW(model.parameters(), lr=args.learning_rate)  # AdamW 优化器

device_type = "cuda" if "cuda" in args.device else "cpu"
print(f'设备类型：{device_type}')
# 根据指定的数据类型设置混精度训练的 dtype，以下步骤为不可缺少的混精度训练准备工作
if args.dtype == 'bfloat16':
    amp_dtype = torch.bfloat16
elif args.dtype == 'float16':
    amp_dtype = torch.float16
else:
    amp_dtype = torch.float32  # 默认为 FP32
print(f'使用混精度训练，数据类型：{amp_dtype}')
# 在 cuda 上启动混精度训练，否则空白上下文
autocast_ctx = nullcontext() if device_type == "cpu" else torch.amp.autocast(device_type='cuda', dtype=amp_dtype) 

设备类型：cuda
使用混精度训练，数据类型：torch.bfloat16


接下来，我们来看看 MiniMind 的训练函数

In [9]:
def train_epoch(epoch, loader, iters, start_step=0, wandb=None):
    start_time = time.time()
    for step, (X, Y, loss_mask) in enumerate(loader, start=start_step + 1):
        # 将输入数据移动到指定设备
        X = X.to(args.device)
        Y = Y.to(args.device)
        loss_mask = loss_mask.to(args.device)

        # 更新学习率
        current_total_step = epoch * iters + step  # 当前训练步数
        total_training_steps = args.epochs * iters  # 总训练步数
        lr = get_lr(current_total_step, total_training_steps, args.learning_rate)
        # lr = get_lr(epoch * iters + step, args.epochs * iters, args.learning_rate) 更新学习率的简化版本
       
        # 将更新后的学习率应用到优化器的参数组中
        for param_group in optimizer.param_groups:
            param_group['lr'] = lr

        # 前向推理和损失计算
        with autocast_ctx:
            res = model(input_ids=X)  # 前向推理，输入为 token ids，输出包含logits，aux_loss和past_key_values
            logits = res.logits  # 模型输出的 logits 得分，形状为 (batch_size, seq_len, vocab_size)
            loss_fct = nn.CrossEntropyLoss(reduction='none')  # reduction='none' 表示不进行平均或求和，保留每个元素的损失值
            # 调整logits和labels形状以匹配交叉熵输入
            shift_logits = logits.reshape(-1, logits.size(-1))  # 重塑为 (batch_size * seq_len, vocab_size)
            shift_labels = Y.reshape(-1)  # 重塑为 (batch_size * seq_len)
            shift_loss_mask = loss_mask.reshape(-1)  # 重塑为 (batch_size * seq_len)
            # 计算基础损失
            raw_loss = loss_fct(shift_logits, shift_labels)  # 计算每个元素的交叉熵损失，形状为 (batch_size * seq_len)
            # 应用loss_mask，只计算需要监督的位置
            masked_loss = (raw_loss * shift_loss_mask).sum() / (shift_loss_mask.sum() + 1e-8)
            # 加上moe的辅助损失（若无moe则aux_loss为0）
            total_loss = masked_loss + (res.aux_loss if res.aux_loss is not None else 0.0)
            # 梯度累积：损失除以累积步数  相当于在显存受限的情况下模拟更大的 batch size
            total_loss = total_loss / args.accumulation_steps

        # 反向传播
        scaler.scale(total_loss).backward()

        if (step + 1) % args.accumulation_steps == 0:
            scaler.unscale_(optimizer)
            torch.nn.utils.clip_grad_norm_(model.parameters(), args.grad_clip)  # 梯度剪裁
            # 更新参数并调整缩放器
            scaler.step(optimizer)
            scaler.update()
            # 清零梯度
            optimizer.zero_grad(set_to_none=True)  # set_to_none=True 可以更高效地清零梯度

        # 日志记录
        if step % args.log_interval == 0 or step == iters - 1:
            spend_time = time.time() - start_time  # 计算已用时间
            current_loss = total_loss.item() * args.accumulation_steps  # 还原为未除以累积步数的损失值
            current_aux_loss = res.aux_loss if res.aux_loss is not None else 0.0
            current_logits_loss = current_loss - current_aux_loss  # 计算仅语言模型部分的损失
            current_lr = optimizer.param_groups[-1]['lr']  # 获取当前学习率
            # 计算剩余时间
            eta_seconds = (spend_time / step) * (iters - step) if step > 0 else 0
            eta_min = eta_seconds // 60
            print(
                f'Epoch:[{epoch + 1}/{args.epochs}]({step}/{iters}), '
                f'loss: {current_loss:.4f}, '
                f'logits_loss: {current_logits_loss:.4f}, '
                f'aux_loss: {current_aux_loss:.4f}, '
                f'lr: {current_lr:.8f}, '
                f'epoch_time: {eta_min:.1f}min'
            )
            # wandb日志（若启用）
            if wandb:
                wandb.log({
                    "loss": current_loss, 
                    "logits_loss": current_logits_loss, 
                    "aux_loss": current_aux_loss, 
                    "learning_rate": current_lr, 
                    "epoch_time": eta_min
                })

        # 到达指定保存步数时，保存模型（仅主进程）
        if args.save_interval > 0 and (step % args.save_interval == 0 or step == iters - 1):
            if not dist.is_initialized() or dist.get_rank() == 0:
                os.makedirs(args.save_dir, exist_ok=True)  # 确保保存目录存在
                model.eval()
                moe_suffix = '_moe' if lm_config.use_moe else ''  # 根据是否使用 MOE 添加后缀
                ckp = f'{args.save_dir}/{args.save_weight}_{lm_config.dim}{moe_suffix}.pth' # checkpoint 文件名包含模型维度、MOE 信息
                # 处理DDP和compile包装的模型，获取原始模型
                raw_model = model.module if isinstance(model, DistributedDataParallel) else model
                raw_model = getattr(raw_model, '_orig_mod', raw_model)  # 获取原始模型，兼容DDP和compile包装
                state_dict = raw_model.state_dict()
                # 保存模型（转为FP16并移到CPU，节省空间）
                torch.save({k: v.half().cpu() for k, v in state_dict.items()}, ckp)
                print(f'模型已保存至：{ckp}')
                model.train()
                del state_dict

        # 清理显存（可选）
        del X, Y, loss_mask, res, total_loss, raw_loss

准备完毕，我们尝试一轮长度 1 个 iter 的训练.

In [10]:
iter_per_epoch = len(train_loader)  # 每轮迭代次数等于 DataLoader 的长度
for epoch in range(args.epochs):
    train_epoch(epoch, train_loader, iter_per_epoch)
print('预训练完成！')

Epoch:[1/5](1/1), loss: 8.9759, logits_loss: 8.9759, aux_loss: 0.0000, lr: 0.00050225, epoch_time: 0.0min
模型已保存至：./output/minimind_pretrain_512.pth
Epoch:[2/5](1/1), loss: 8.0780, logits_loss: 8.0780, aux_loss: 0.0000, lr: 0.00037725, epoch_time: 0.0min
模型已保存至：./output/minimind_pretrain_512.pth
Epoch:[3/5](1/1), loss: 7.4790, logits_loss: 7.4790, aux_loss: 0.0000, lr: 0.00022275, epoch_time: 0.0min
模型已保存至：./output/minimind_pretrain_512.pth
Epoch:[4/5](1/1), loss: 7.1507, logits_loss: 7.1507, aux_loss: 0.0000, lr: 0.00009775, epoch_time: 0.0min
模型已保存至：./output/minimind_pretrain_512.pth
Epoch:[5/5](1/1), loss: 6.9811, logits_loss: 6.9811, aux_loss: 0.0000, lr: 0.00005000, epoch_time: 0.0min
模型已保存至：./output/minimind_pretrain_512.pth
预训练完成！


In [11]:
del model